<a href="https://colab.research.google.com/github/adityajain2407/aj-biz-cases/blob/main/AJ_Yulu_Review_HT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement:

In [ ]:
'''
Based on the concept of Demand forecasting - what are peak hours, when is the right time to charge, which season will be drop/peak to manage
availabiliyt of bikes


Select an appropriate test to check whether:
Working Day has effect on number of electric cycles rented (count) - 2 sample independent t-test
-cant be anova as need 2+ series- here we have only 2 1/0
- no z-test as need std devi of population
- cant be chi-square as need both to be cat- using proprtions --> test of independence - crosstab, proortions,
- here it is cat and numerical (count)

No. of cycles rented similar or different in different seasons - num and cat - more than 2 seasons - ANOVA
No. of cycles rented similar or different in different weather - - more than 2 seasons - ANOVA
Weather is dependent on season (check between 2 predictor variable) - both cat --> chi-square

Assumtions for ANOVA
1- check normality
2- equal variance among each group

even if assumptoins fail= still go ahead and try it out for now

Whatever insights we get is for the sample data, but we need HT to understand if thats true for Population or not
1. EDA
2. HT

Each row represents 1 hr of data
object --> Datetime (not imp)
No missing data
no duplicates
Sampling done is good - as for every season almost same no. of hours is given in data - no bias to any season
But for weather it is uneven - there is outliser for weather 4 - can ignore this weatyher for analysis
As cat are also given numbers (encoded), we can check relationship between all using heatmap
extra- to capture non-linear relaitonship between vartianbles --> using spearmen correaltion (default - pearson works for liner relaitonship - Google)
Target variable is count - need to check what vaariables impacts it --> temp, humidity
But as working day, holiday ,etc are not giving exact intuition and its a grey area - we can check bivariate analysis to get more insights
Why to chose median over mean? Outlier robust eg
We can confirm the correaltion suing box plot - as median of diff seasons are close - not much effect
but for weather 3, its diff from rest - so some impact

We can see there are outlisers - not a good idea to direclty remove them - as not aware of the reasoning - is it due to
data collection error, data sampling error or natrual outliers. If natrual - removing it will be like manipulating the data - incorrect
so we will transform it to Normal -- why - bcoz most of the tests assume distribution to be normal (parametric tests) -
extra 0- log transform is reversible - we can get back original data -we are not permanently changing it


Testing for normality was not reqd - as we can asusme and go ahead with analysis

From IQR - we can see how many outlisers and as 300 which is less % of overall, if we knew the reason and not natural out;lsers
we can easlity reamove the outlisers

So far, we have done EDA on sample - for now, season weather, temp and humidity has impact,
workingday, holiday has no impact --<

Now, lets do HT over it

'''

'\nBased on the concept of Demand forecasting - what are peak hours, when is the right time to charge, which season will be drop/peak to manage\navailabiliyt of bikes\n\n\nSelect an appropriate test to check whether: \nWorking Day has effect on number of electric cycles rented (count) - 2 sample independent t-test\n-cant be anova as need 2+ series- here we have only 2 1/0 \n- no z-test as need std devi of population\n- cant be chi-square as need both to be cat- using proprtions --> test of independence - crosstab, proortions, \n- here it is cat and numerical (count)\n\nNo. of cycles rented similar or different in different seasons - num and cat - more than 2 seasons - ANOVA\nNo. of cycles rented similar or different in different weather - - more than 2 seasons - ANOVA\nWeather is dependent on season (check between 2 predictor variable) - both cat --> chi-square\n\nAssumtions for ANOVA\n1- check normality \n2- equal variance among each group\n\neven if assumptoins fail= still go ahead a

In [ ]:
# Importing required libraries -
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_ind # T-test for independent samples
from scipy.stats import shapiro # Shapiro-Wilk’s test for Normality
from scipy.stats import levene # Levene's test for Equality of Variance
from scipy.stats import f_oneway # One-way ANOVA
from scipy.stats import chi2_contingency # Chi-square test of independence
# from scipy.stats import kstest # KS test for Normality

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


link = 'https://drive.google.com/file/d/1o94fXnmvrx6jRgI6S-SeZ3tfnKjCDY0i/view?usp=sharing'

id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('bike_sharing.csv')

NameError: ignored

In [ ]:
# Loading the dataset -
df=pd.read_csv('bike_sharing.csv')
df.sample(10)

In [ ]:
# Shape of the dataset -
print("No. of rows : ", df.shape[0])

In [ ]:
df.info()

In [ ]:
# Checking for null values -
(df.isna().sum() / len(df)) * 100

In [ ]:
# Checking for duplicate rows -
dup_rows = df[df.duplicated()]
print("No. of duplicate rows: ", dup_rows.shape[0])

In [ ]:
def dist_check(df, col_name):
  print("Unique values : ", df[col_name].unique())
  print("Value counts : ")
  print(df[col_name].value_counts())

In [ ]:
col_list = ['workingday',	'holiday',	'weather', 'season']
for col in col_list:
  print(col, " -")
  dist_check(df, col)
  print("\n")

In [ ]:
# Correlation Heatmap -

plt.figure(figsize=(12, 7))
sbn.heatmap(df.corr(method='spearman'),
            annot=True, cmap='viridis')
plt.show()

from the correlation we can verify some logical points:
- feeling temperature or aparent temprature and temp are highly correlated, because they are most of the times approximately the same have a very small diffrerence
- count, causal, registered are all correlated to each other because all of them

In [ ]:
# Dropping highly correlated columns -
dfn = df.drop(columns=['casual', 'registered', 'atemp'])

In [ ]:
# Outlier Detection using Boxplots -

sbn.set(style="whitegrid")
fig = plt.figure(figsize=(8, 25))
fig.subplots_adjust(right=1.5)

for plot in range(1, len(col_list)+1):
    plt.subplot(5, 2, plot)
    sbn.boxplot(x=dfn[col_list[plot-1]], y=dfn['count'])

plt.show()

In [ ]:
# Checking distribution of 'count' column -
plt.figure(figsize=(14, 5))

#Histogram
plt.subplot(1, 2, 1)
sbn.distplot(dfn['count'], bins=10)

#Boxplot
plt.subplot(1, 2, 2)
sbn.boxplot(y=dfn['count'])
plt.title('Boxplot')

plt.show()

We can see that outliers are present in the given columns. We need to figure out a way to deal with them before starting with the tests.

We have multiple options available on how to proceed with these outlier values.

1. Try to understand if these values make any sense according to the business problem. If yes, then we can keep them as it is.
2. In case these outliers are some invalid values which do not make much sense, we can remove them using the IQR.
3. Or we can apply a log transformation on the data to reduce the effect of these outliers.

1. -
* The outliers in the given data set are the no. of bike rides per session/day. These values could sometimes be higher than expected due to increase in the crowd on certain days/occasions.
* These data values are important for capturing variations in the data. Hence, in this case, the ideal approach of dealing with outliers would be to leave them as it is.
* But since the tests that we are going to apply are based on the assumption that the dataset is normal or near normal, we will drop those outlier values using the IQR method.

In [ ]:
# 2.
# Checking distribution after applying log transformation -
plt.figure(figsize=(14, 5))

#Histogram
plt.subplot(1, 2, 1)
sbn.distplot(np.log(dfn['count']), bins=10)

#Boxplot
plt.subplot(1, 2, 2)
sbn.boxplot(y=np.log(dfn['count']))
plt.title('Boxplot')

plt.show()

In [ ]:
# # 3.
# # Outlier Treatment using IQR (not needed but, we can do it) -

# q1 = dfn['count'].quantile(0.25)
# q3 = dfn['count'].quantile(0.75)
# iqr = q3-q1

# dfn = dfn[(dfn['count']>(q1-1.5*iqr) ) & (dfn['count']<(q3+1.5*iqr))]

# print("No. of rows : ", dfn.shape[0])

#### Aggregating the total no. of bike rides based on the given factors -

In [ ]:
# 1. Workingday -
pd.DataFrame(dfn.groupby('workingday')['count'].describe())

In [ ]:
# 2. Holiday -
pd.DataFrame(dfn.groupby('holiday')['count'].describe())

In [ ]:
# 3. Season -
pd.DataFrame(dfn.groupby('season')['count'].describe())

In [ ]:
# 4. Weather -
pd.DataFrame(dfn.groupby('weather')['count'].describe())

In [ ]:
#we ask ourselves such doubts in industry to get direction of analyssi
#can do independent t-test : we are doing one tailed
'''
Extra - we no. of data points are large ? t test converges to z-test

Most of the statistical packages  by default does t-test only
'''

---



### Ques. 1 - Is there any significant difference between the no. of bike rides on weekdays and weekends?
---

#### **Step 1:** Define the null and alternate hypothesis

$H_0:$ The demand of bikes on weekdays is greater or similar to the demand of bikes on weekend.

$H_a:$ The demand of bikes on weekdays is less than the demand of bikes on weekend.

Let $\mu_1$ and $\mu_2$ be the average no. of bikes rented on weekdays and weekends respectively.

Mathematically, the above formulated hypothesis can be written as:

$H_0: \mu_1 >= \mu_2$

$H_a: \mu_1 < \mu_2$

**Ques.** What is the difference between a t-test and a z-test?

**Ans.**

* A t-test looks at two sets of data that are different from each other, with no standard deviation or variance.

* A z-test views the averages of data sets that are different from each other but have the standard deviation or variance given.

* The t test as compared with z test has its advantage for small sample comparison. As n increases, t approaches to z. The advantage of t test disappears, and t distribution simply becomes z distribution.

* In other words, with large n, t test is just close to z test and one doen't lose anything to continue to use t test.

* In the past, for convenience, we use z table when n > 30.  We don't have to do it anymore.

* In fact, all statistical packages use t test even n is large. This is easy, convenience with computer programming, and is correct. All statistical packages are good references.

#### **Step 2:** Select an appropriate test

Note that the standard deviation of the population is not known.

In [ ]:
'''
we keep equal number to make sure we dont introduce variance - follow this practice

In [ ]:
weekday = dfn[dfn['workingday'] == 1]['count'].sample(2999)
weekend = dfn[dfn['workingday'] == 0]['count'].sample(2999)

**Ques.** Why do we take same no. of samples from two different populations for conducting the tests?

**Ans.**
* Unequal sample sizes can lead to unequal variances between samples, which affects the assumption of equal variances in tests like t-test, ANOVA, etc.

* Having both unequal sample sizes and variances dramatically affects the statistical power of a test.


In [ ]:
print('The sample standard deviation of the bike rides on weekday is:', round(weekday.std(), 2))
print('The sample standard deviation of the bike rides on weekend is:', round(weekend.std(), 2))
#unequal variance

As the sample standard deviations are different, the population standard deviations can be assumed to be different.

This is a one-tailed test concerning two population means from two independent populations. As the population standard deviations are unknown, the two sample independent t-test will be the appropriate test for this problem.

#### **Step 3:** Decide the significance level

As given in the problem statement, we select α = 0.05.

In [ ]:
alpha = 0.05

#### **Step 4:** Calculate the p-value

In [ ]:
def result(p_value, alpha):
  if p_value < alpha:
    print(f'As the p-value {p_value} is less than the level of significance, we reject the null hypothesis.')
  else:
    print(f'As the p-value {p_value} is greater than the level of significance, we fail to reject the null hypothesis.')

In [ ]:
test_stat, p_value = ttest_ind(weekday, weekend, equal_var=False, alternative='less') #two-tailed, greater - extra -
print('The p-value is : ', p_value)
#you can get diff p value depending on teh pre-processing done
result(p_value, alpha)

**Observation:** Since the p-value is greater than the 5% significance level, we fail to reject the null hypothesis. Hence, we have enough statistical evidence to say that the average no. of bike rides during weekdays is greater than or equal to those on weekends.

---
### Ques. 2 - Is there any significant difference between the no. of bike rides on regular days and holidays?
---

#### **Step 1:** Define the null and alternate hypothesis

$H_0:$ The demand of bikes on regular days is greater or similar to the demand of bikes on holidays.

$H_a:$ The demand of bikes on regular days is less than the demand of bikes on holidays.

Let $\mu_1$ and $\mu_2$ be the average no. of bikes rented on regular days and holidays respectively.

Mathematically, the above formulated hypothesis can be written as:

$H_0: \mu_1 >= \mu_2$

$H_a: \mu_1 < \mu_2$

#### **Step 2:** Select an appropriate test

Again the standard deviation of the population is not known.

In [ ]:
holiday = dfn[dfn['holiday'] == 1]['count'].sample(299)
regular = dfn[dfn['holiday'] == 0]['count'].sample(299)

In [ ]:
print('The sample standard deviation of the bike rides on holidays is:', round(holiday.std(), 2))
print('The sample standard deviation of the bike rides on regular days is:', round(regular.std(), 2))

As the sample standard deviations are different, the population standard deviations can be assumed to be different.

This is also a one-tailed test concerning two population means from two independent populations. As the population standard deviations are unknown, the two sample independent t-test will be the appropriate test for this problem.

#### **Step 3:** Decide the significance level

The significance level (α) is already set to 5% i.e., 0.05

#### **Step 4:** Calculate the p-value

In [ ]:
test_stat, p_value = ttest_ind(regular, holiday, equal_var=False, alternative='less')
print('The p-value is : ', p_value)

result(p_value, alpha)

**Observation:** Since the p-value is greater than the 5% significance level, we fail to reject the null hypothesis. Hence, we have enough statistical evidence to say that the average no. of bike rides during regular days is greater than or equal to those on holidays.

---
### Ques. 3 - Is the demand of bicycles on rent same for different weather conditions?
---

#### **Step 1:** Define the null and alternate hypothesis

$H_0:$ The average no. of bike rides in different weather conditions  are equal.

$H_a:$ The average no. of bike rides in different weather conditions are not equal.

Let $\mu_1$ and $\mu_2$ be the average no. of bikes rented on weekdays and weekends respectively.

#### **Step 2:** Select an appropriate test


In [ ]:
dfn = dfn[~(dfn['weather']==4)] #due to outlier above

In [ ]:
#we have to check if all 3 series are close or not - why 750 - because if do descibe 850 in w3 so need to be between 30 and 850
#we will do ANOVA - now need to check normality and populaiotn variaance of all group equal - need to vlaidate assumptions
#even if normlaity fails, we can still go ahead but variance thing should be true
w1 = dfn[dfn['weather'] == 1]['count'].sample(750)
w2 = dfn[dfn['weather'] == 2]['count'].sample(750)
w3 = dfn[dfn['weather'] == 3]['count'].sample(750)

In [ ]:
dfn.groupby(['weather'])['count'].describe()

This is a problem, concerning three independent population means. **One-way ANOVA** could be the appropriate test here provided normality and equality of variance assumptions are verified.

The ANOVA test has important assumptions that must be satisfied in order for the associated p-value to be valid.

* The samples are independent.
* Each sample is from a normally distributed population.
* The population variance of the groups are all equal.

Now, we will be using the following statistical tests to check the normality and euality of variance of the data set -

* For testing of normality, Shapiro-Wilk’s test is applied to the response variable.

* For equality of variance, Levene test is applied to the response variable.

#### Shapiro-Wilk’s test -

We will test the null hypothesis

>$H_0:$ Count follows normal distribution

against the alternative hypothesis

>$H_a:$ Count doesn't follow normal distribution



In [ ]:
# Assumption 1: Normality

w, p_value = shapiro(dfn['count'].sample(4999))
print('The p-value is : ', p_value)

result(p_value, alpha)
#not normal - but still as lots of data points - by CLT - we go ahead with assumption of normal

#### Levene’s test -

We will test the null hypothesis

>$H_0$: All the count variances are equal

against the alternative hypothesis

>$H_a$: At least one variance is different from the rest

In [ ]:
#Assumption 2: Homogeneity of Variance - this test is to check if gorups have equal variance or not

stat, p_value = levene(w1, w2, w3)
print('The p-value is : ', p_value)

result(p_value, alpha)

#we can also check from describe std data - so we cant apply ANOVA
#HW try Kruskal-Wallis H-test

#Extra: Parametyric - we assume distri to be normal to apply them, non-parametric - no such requirement of normality
#we use parametric first as if its normal we get great insights - if doenst work, we go for non-parametric as they are more robust
#due to no assumptions

cheat sheet for all tests: https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/

**Note:** If these assumptions are not true for a given set of data (like in this case), it may still be possible to use the **Kruskal-Wallis H-test** or the **Alexander-Govern test** although with some loss of power.

### Central Limit Theorem -

You all must have studies about the CLT in previous classes.

* According to this theorem, the distribution of sample means approximates a normal distribution as the sample size gets larger, regardless of the population's distribution.

* In other words, if we find the mean of a large number of independent random variables, the mean
will follow a normal distribution, irrespective of the distribution of the original variables.

* In practice, sample sizes equal to or greater than 30-40 are often considered sufficient for the CLT to hold.

Hence, the sample size being large enough, we don't need to worry about the non-normality of distribution of the data set in hand before applying the tests.

Eventually, as the sample size gets larger, the distribution of sample means will fall into a normal or near normal shape.


**Ques.** What are some of the basic methods (other than statistical tests) to test the normality & homogeneity of variance?

A. To check for **Normality -**

In [ ]:
# Method 1: Using a Histogram
# You should see a 'Bell' shaped curve.
sbn.distplot(dfn['count'].sample(4999))

In [ ]:
# Method 2: Using a Q-Q plot
# The linearity of points suggests that the data is normally distributed.
stats.probplot(dfn['count'], dist='norm', fit=True, plot=plt)
plt.title('Q-Q Plot')
plt.show()

In [ ]:
# Method 3: Check skewness & kurtosis
# Skewness should be close to 0 and Kurtosis close to 3.

print("Skewness : ", df['count'].skew())
print("Kurtosis : ", df['count'].kurt())

In [ ]:
## Method 4: Using KS Test to compare with the Gaussian CDF
zs = (dfn["count"] - dfn["count"].mean())/dfn["count"].std()
stats.kstest(zs, stats.norm.cdf)

B. To check for **Homogeneity of Variance -**

In [ ]:
# Method 1:
print(w1.var(), w2.var(), w3.var())

#### **Step 3:** Decide the significance level

The significance level (α) is already set to 5% i.e., 0.05

#### **Step 4:** Calculate the p-value

One-way ANOVA -


In [ ]:
test_stat, p_value = f_oneway(w1, w2, w3)
print('The p-value is : ', p_value)

result(p_value, alpha)

**Observation:** Since the p-value is less than the 5% significance level, we reject the null hypothesis. Hence, we have enough statistical evidence to say that the average no. of bike rides in different weather conditions are not equal.

---
### Ques. 4 - Is the demand of bicycles on rent same for different seasons?
---

#### **Step 1:** Define the null and alternate hypothesis

$H_0:$ The average no. of bike rides in different seasons  are equal.

$H_a:$ The average no. of bike rides in different seasons are not equal.

#### **Step 2:** Select an appropriate test

In [ ]:
s1 = dfn[dfn['season'] == 1]['count'].sample(2399)
s2 = dfn[dfn['season'] == 2]['count'].sample(2399)
s3 = dfn[dfn['season'] == 3]['count'].sample(2399)
s4 = dfn[dfn['season'] == 3]['count'].sample(2399)

In [ ]:
dfn.groupby(['season'])['count'].describe()

#### **Step 3:** Decide the significance level

The significance level (α) is already set to 5% i.e., 0.05

#### **Step 4:** Calculate the p-value

We have already performed tests for normality and homogeneity of variance. So we will be directly moving onto the One-way ANOVA test.

In [ ]:
test_stat, p_value = f_oneway(s1, s2, s3, s4)
print('The p-value is : ', p_value)

result(p_value, alpha)

**Observation:** Since the p-value is less than the 5% significance level, we reject the null hypothesis. Hence, we have enough statistical evidence to say that the average no. of bike rides in different seasons are not equal.

**Ques.** How does the increase in sample size affect hypothesis testing?

**Ans.** Increasing sample size makes the hypothesis test more sensitive, more likely to reject the null hypothesis when it is, in fact, false. Thus, it increases the power of the test.

In [ ]:
#checking dependcy of 1 cat variable over another - chi-square test of independence - non-parametric - no requirement for normal distri

---
### Ques. 5 -  Are the weather conditions significantly different during different seasons?
---



#### **Step 1:** Define the null and alternate hypothesis

$H_0:$ Weather conditions are independent of the season.

$H_a:$ Weather condition depends on the ongoing season.

#### **Step 2:** Select an appropriate test

Although the data values in 'season' and 'weather' columns are numerical, as per our intuition, they still represent different catgories. Hence, we will encode them accordingly before moving onto the tests.

In [ ]:
dict1 = {1: 'Sunny',
         2: 'Cloudy',
         3: 'Rainy'}
dfn['weather_enc'] = dfn['weather'].map(dict1)

In [ ]:
dict2 = {1: 'Summer',
         2: 'Monsoon',
         3: 'Winter',
         4: 'Autumn'}
dfn['season_enc'] = dfn['season'].map(dict2)

Here we will be comparing two different categorical variables, 'season' and 'weather'. So will perform a **Chi-square test**.

In [ ]:
contigency= pd.crosstab(dfn.season_enc, dfn.weather_enc)
contigency

In [ ]:
contigency.plot(kind='bar')

#### **Step 3:** Decide the significance level

The significance level (α) is already set to 5% i.e., 0.05

#### **Step 4:** Calculate the p-value

In [ ]:
chi2, pval, dof, exp_freq = chi2_contingency(contigency, correction=False) #HW - read about correction param - and the history behind it
print('Chi-square Statistic: {} \n P-value: {} \n Degree of Freedom: {} \n Expected Frequencies: {}'.format(chi2, pval, dof, exp_freq))

In [ ]:
result(pval, alpha)

**Observation:** Since the p-value is less than the 5% significance level, we reject the null hypothesis. Hence, we have enough statistical evidence to say that the weather conditions are dependent on the ongoing season.

### Insights and Recommendations

#### **EDA based insights -**

1. Total 10,886 rows were present in the data set.
2. Neither missing values, nor duplicate rows were found.
3. 'temp' and 'atemp' columns were found to be highly correlated. </br> Dropping one of them (atemp) to avoid multicollinearity.
4. 'count', 'casual' and 'registered' columns were highly correlated. </br> Dropping casual & registered columns to avoid multicollinearity.
5. Outlier values were found in the 'count' column.

#### **Insights from hypothesis testing -**
1. The no. of bikes rented on weekdays is comparatively higher than on weekends.
2. The no. of bikes rented on regular days is comparatively higher than on holidays.
2. The demand of bicycles on rent differs under different weather conditions.
3. The demand of bicycles on rent is different during different seasons.
4. The weather conditions are surely dependent upon the ongoing season.

#### **Miscellaneous observations -**
The distribution of 'count' column wasn't actually normal or near normal. </br> Infact the column's distribution is found to be a bit skewed towards right.

#### **Generic recommendations -**

* The demand of bikes on rent are usually higher during Weekdays.
* The demand of bikes on rent are usually higher during Regular days.
* The chances of person renting a bike are usually higher during Season 3.
* The chances of person renting a bike are usually higher during Weather condition 1.

We recommend the company to maintain the bike stocks accordingly.

